Requirements 

- Tensorflow, if you have anaconda `conda install tensorflow`
- `pip install -y pandas numpy keras`

In [359]:
# Data
import pandas as pd
import numpy as np
import sys
import re
import os

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

## Inputs

In [2]:
df = pd.read_csv('data/tweets-fixed.tsv', sep='\t', index_col=0)
df.head(2)

,source,text,created,retweets,favorites,is_retweet,id
0,Twitter for iPhone,RT @GOPChairwoman: .@realDonaldTrump is the Pa...,12-14-2017 23:26:54,4262,0,True,941449449850761217
1,Twitter for iPhone,“Manufacturing Optimism Rose to Another All-Ti...,12-14-2017 21:20:51,4789,19906,False,941417725833998340


# Processing

Sanitize tweets into a format suitable for training

In [293]:
# Build a set of characters to remove to reduce our vocab set
chars_to_remove = {'"', '$', '%', "'", '(', ')', '*', '+', '/', ';', 
                   '<', '=', '>', '[', '\\', ']', '_', '`',  '{', '|', '}', '~'}
                    #'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '#',
# In 32,000 tweets he apparently never uses a comma, probably an artifact of CSV export.    

# TODO: Order tweets

text = ''
for i, tweet in enumerate(df[df.is_retweet == False].text):
    # print tweet
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Fix ampersand HTML artifact
    tweet = tweet.replace('&amp;', '&')
    
    # lower() to reduce pool of possible characters (lower-case's strings)
    # the decode/encode step is to remove non-ascii characters like 
    tweet = tweet.lower().decode("ascii", errors="ignore").encode()

    # Remove chars from our chars_to_remove set with list comprehension
    for x in chars_to_remove:
        tweet = tweet.replace(x, ' ')
    #tweet = ''.join([x for x in tweet if x not in chars_to_remove]).rstrip()
    
    # If there's no tweet left (just a URL for example)
    if not tweet:
        continue
        
    # split and rejoin for odd spacing
    tweet = ' '.join(tweet.split())        
    
    # Add period if no ending line or a space
    if not tweet[-1] in ['.', '!', '?']:
        tweet = tweet + '. '
    else:
        tweet += ' '

    # Add to text (by redefining with +=)
    text += tweet
    
    #print tweet, '\n'
    #print
    # if i > 100: break
    
len(text)

3306514

Create character set and mapping

In [295]:
chars = sorted(list(set(text)))
print 'Total chars: {}'.format(len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total chars: 45


Cut text up into arbitrary sequences using a maximum length and a step size

In [241]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

('nb sequences:', 1102158)


Vectorization of inputs

- x: has dimensionality (num_sequences, maxlength, num_chars)
- y: has dimensionality (num_sequences, num_chars)

A sequence in `x` is represented as a 2d matrix of `maxlength` by `num_chars`. This means each row corresponds to what character in the possible character set, with every value being 0 except for a 1 in the position corresponding to the character. For example, the letter 'a' corresponds to the 19th position (column) in the matrix. 

In [304]:
# Example from above
char_indices['a']

19

In [247]:
# Start by creating arrays of zeros with our final dimensionality
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Model

We'll build a simple LSTM model to start with that will likely need to be tweaked to get better results

In [248]:
# Simplest model abstraction is Sequential
model = Sequential()
# Add LSTM layer with 256 memory units
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
# Add Dropout layer at 20% node dropout (avoids overfitting)
model.add(Dropout(0.2))
# Add final Dense layer which corresponds to each of our characters
model.add(Dense(len(chars), activation='softmax'))
# Compile the model 
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training

We'll use checkpointing and select the final set of weights with the lowest loss

In [358]:
ModelCheckpoint?

In [72]:
# Make directory for model output
if not os.path.exists('model'):
    os.mkdir('model')

path = "model/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Fit the model

In [ ]:
model.fit(x, y, epochs=20, batch_size=128, callbacks=callbacks_list)

# Text Generation

Load model with weights corresponding to smallest loss

In [309]:
os.listdir('model')

['weights-improvement-20-1.2692.hdf5']

In [310]:
weights = os.path.join('model', os.listdir('model')[0])  # Grab weight in model dir
model.load_weights(weights)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Pull out random seed

In [350]:
start_index = np.random.randint(0, len(text) - maxlen - 1)

generated = ''
sentence = text[start_index: start_index + maxlen]
generated += sentence
print 'Generating with seed: {}\n'.format(sentence)

Generating with seed: & russia. great night in iowa - special 



Generate text from seed

In [351]:
def str_to_input(sentence, maxlen, chars, char_indices):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
    return x_pred

In [353]:
sys.stdout.write(generated)
for i in range(400):
    # Convert text seed to input
    x_pred = str_to_input(sentence, maxlen, chars, char_indices)
    
    # Make prediction with model
    preds = model.predict(x_pred, verbose=0)[0]
    next_char = indices_char[np.argmax(preds)]

    # Generate text
    generated += next_char
    sentence = sentence[1:] + next_char
    #sys.stdout.write(next_char)
    #sys.stdout.flush()

& russia. great night in iowa - special course in the u.s. in the world s starting to the presidential republicans are a great presidential and the real man and the world s strong and start to the presidential republicans are a great presidential and the real man and the world s strong and start to the presidential republicans are a great presidential and the real man and the world s strong and start to the presidential republicans are a [  2.73892860e-04   1.39975906e-04   3.91857931e-03   2.14983523e-03
   1.23859849e-03   3.47654248e-04   1.93723354e-05   2.35167914e-03
   1.11452281e-03   1.67771461e-04   2.32513470e-04   4.05182102e-04
   7.20372118e-05   1.64037338e-04   1.02123391e-04   1.08421336e-04
   1.72666605e-05   3.11164367e-05   4.16096766e-03   2.10555628e-01
   3.49141285e-02   2.49546021e-02   3.07344217e-02   1.66234393e-02
   4.43036482e-02   4.46878523e-02   3.08000762e-02   4.74325530e-02
   1.05540482e-02   4.00115922e-03   2.75060702e-02   2.39861310e-02
   5.5